In [1]:
import pandas as pd

df_income = pd.read_excel("./data/expenses.xlsx", sheet_name="Income")
df_expenses = pd.read_excel("./data/expenses.xlsx", sheet_name="Expenses")

In [2]:
# Droping some columns that were unnecessary..
col_to_drop = [
    "Transaction amount in transaction currency",
    "Transaction currency",
    "Amount in account currency",
    "Account currency"
]
df_income.drop(columns=col_to_drop, inplace=True)
df_expenses.drop(columns=col_to_drop, inplace=True)


# adding type to each df 
df_income["Type"] = "Income"
df_expenses["Type"] = "Expenses"

In [3]:
# Joining these two dataframes of expenses and income into one 
df = pd.concat([df_income, df_expenses], ignore_index=True)
# Renaming some columns for better understanding and prediciton
df = df.rename(columns={
    'Date and time': 'Date', 
    'Amount in default currency': 'Amount',
    'Default currency' : 'Currency'
    })
df

,Date,Category,Account,Amount,Currency,Comment,Type
0,"March 31, 2024",Balancing,Main,16.00,CAD,NaN,Income
1,"March 28, 2024",Paycheck,Main,1427.19,CAD,NaN,Income
2,"March 15, 2024",Paycheck,Main,1427.19,CAD,NaN,Income
3,"March 3, 2024",Balancing,Main,6.00,CAD,NaN,Income
4,"March 1, 2024",Paycheck,Main,1427.19,CAD,NaN,Income
...,...,...,...,...,...,...,...
193,"May 5, 2023",Transportation,Main,9.68,CAD,NaN,Expenses
194,"May 5, 2023",Transportation,Main,4.46,CAD,NaN,Expenses
195,"May 5, 2023",Withdrawal,Main,300.00,CAD,NaN,Expenses
196,"May 3, 2023",Groceries,Main,74.52,CAD,Utensils and some snacks,Expenses
